In [ ]:
import os

os.chdir('../')

In [6]:
%cd end-to-end-chest-cancer-classification/

[WinError 2] The system cannot find the file specified: 'end-to-end-chest-cancer-classification/'
e:\Github repositories\end-to-end-chest-cancer-classification


e:\Jupyter\envs\eeccc\lib\site-packages\IPython\core\magics\osm.py:393: UserWarning: using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [7]:
from  dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_learning_rate: float
    params_is_augmentation: bool
    params_image_size: list

In [8]:
from ChestClassifier.constants import *
from ChestClassifier.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_trainig_config(self):
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, 'Chest-CT-Scan-data')
        create_directories([training.root_dir])

        return TrainingConfig(
            root_dir=training.root_dir,
            trained_model_path=training.trained_model_path,
            updated_base_model_path=prepare_base_model.updated_base_model_path,
            training_data=training_data,
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_learning_rate=params.LEARNING_RATE,
            params_image_size=params.IMAGE_SIZE,
            params_is_augmentation=params.AUGMENTATION
        )

In [9]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [14]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)

    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            class_mode="sparse",
            **dataflow_kwargs
        )
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            class_mode="sparse",
            **dataflow_kwargs
        )
        print(f"Train generator batch size: {self.train_generator.batch_size}")
        print(f"Valid generator batch size: {self.valid_generator.batch_size}")



    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [15]:
try:
    config = ConfigurationManager()
    training_config = config.get_trainig_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2024-11-15 20:40:19,147: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-11-15 20:40:19,153: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-15 20:40:19,157: INFO: common: created directory at: artifacts]
[2024-11-15 20:40:19,158: INFO: common: created directory at: artifacts/training]
Found 68 images belonging to 2 classes.
Found 275 images belonging to 2 classes.
Train generator batch size: 32
Valid generator batch size: 32
Epoch 1/10
8/8 [==============================] - 14s 2s/step - loss: 0.8395 - accuracy: 0.6016 - val_loss: 0.7111 - val_accuracy: 1.0000
Epoch 2/10
8/8 [==============================] - 9s 1s/step - loss: 0.6738 - accuracy: 0.6543 - val_loss: 0.6575 - val_accuracy: 0.1406
Epoch 3/10
8/8 [==============================] - 9s 1s/step - loss: 0.5106 - accuracy: 0.5267 - val_loss: 0.4478 - val_accuracy: 0.6094
Epoch 4/10
8/8 [==============================] - 9s 1s/step - loss: 0.4818 - accuracy: 0.7243 - val_loss: 0.3896 -